# SIMCCT User Analytics


In [50]:
# imports
from os import environ as env

from bson.son import SON
from pymongo import MongoClient
import pandas as pd

In [33]:
conn = MongoClient(env.get('MONGO_URI'))

db_name = 'arc_dev'
collection = 'users'
db = conn[db_name]

In [34]:
db

Database(MongoClient(host=['mongodb:27017'], document_class=dict, tz_aware=False, connect=True), 'arc_dev')

In [62]:
cursor = db[collection].find({'profile': {'$exists': True}}, projection={'password': 0, '_id': False})
df = pd.DataFrame(list(cursor))

In [63]:
df.head()

,email,first_name,last_name,profile,admin_profile,saved_alloys,active,admin,disable_admin,verified,created,last_updated,ratings,login_data
0,ironman@avengers.io,Tony,Stark,"{'aim': 'Experimentation', 'highest_education'...",{'position': 'Genius Playboy Billionaire Phila...,"[{'_id': 5d95e4dd2260c5f14fd96b10, 'name': 'Ar...",True,True,False,True,2019-10-03 12:09:00.907,2019-10-03 12:09:00.907,[],[]
1,black_widow@marvel.io,Natasha,Romanoff,"{'aim': 'Engineering Work', 'highest_education...",NaN,"[{'_id': 5d95e4dd2260c5f14fd96b16, 'name': 'Ar...",True,False,False,True,2019-10-03 12:09:00.907,2019-10-03 12:09:00.907,[],[]


In [65]:
pipeline = [
    {'$unwind': '$profile'},
    {'$project': {'profile': 1, '_id': False}},
]

res = db[collection].aggregate(pipeline)
list(res)

[{'profile': {'aim': 'Experimentation',
   'highest_education': 'Masters Degree',
   'sci_tech_exp': 'Advanced',
   'phase_transform_exp': 'Advanced'}},
 {'profile': {'aim': 'Engineering Work',
   'highest_education': 'High School',
   'sci_tech_exp': 'Advanced',
   'phase_transform_exp': 'Beginner'}}]

In [74]:
pipeline = [
    {'$unwind': '$profile'},
    {'$project': {
        'aim': '$profile.aim', 
        'highest_education': '$profile.highest_education', 
        'sci_tech_exp': '$profile.sci_tech_exp', 
        'phase_transform_exp': '$profile.phase_transform_exp',
        '_id': 0
        }
    },
]

res = db[collection].aggregate(pipeline)
profile_df = pd.DataFrame(list(res))
profile_df

,aim,highest_education,sci_tech_exp,phase_transform_exp
0,Experimentation,Masters Degree,Advanced,Advanced
1,Engineering Work,High School,Advanced,Beginner


,aim,highest_education,sci_tech_exp,phase_transform_exp
0,Experimentation,Masters Degree,Advanced,Advanced
1,Engineering Work,High School,Advanced,Beginner
